In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [73]:
data1 = pd.read_csv('../fangchan/lianjia1.csv', encoding='gbk')
data2 = pd.read_csv('../fangchan/lianjia2.csv', encoding='gbk')
data3 = pd.read_csv('../fangchan/lianjia3.csv', encoding='utf-8')
data4 = pd.read_csv('../fangchan/lianjia4.csv', encoding='utf-8')
data5 = pd.read_csv('../fangchan/lianjia5.csv', encoding='utf-8')
data6 = pd.read_csv('../fangchan/lianjia6.csv', encoding='utf-8')
data7 = pd.read_csv('../fangchan/lianjia7.csv', encoding='utf-8')

In [74]:
data = pd.concat([data1, data2, data3, data4, data5, data6, data7])
data = data.dropna()

In [75]:
data = data[data.cjshijian.str.contains('2015-')]

In [76]:
data = data[['cjdanjia', 'cjxiaoqu', 'cjlouceng']]

In [77]:
data['cjdanjia'] = data.cjdanjia.str.replace('元/平', '').astype(np.float32).map(lambda x: round(x/10000, 2))

In [78]:
sum(data.cjxiaoqu.str.split().map(len) != 3)

3

In [79]:
data = data[data.cjxiaoqu.str.split().map(len) == 3]

In [80]:
data = data.assign(xiaoqu=data.cjxiaoqu.map(lambda x:x.split()[0]))

In [81]:
data = data.assign(huxing=data.cjxiaoqu.map(lambda x:x.split()[1]))

In [82]:
data = data.assign(mianji=data.cjxiaoqu.map(lambda x:x.split()[2]))

In [83]:
del data['cjxiaoqu']

In [84]:
sum(data.cjlouceng.str.split('/').map(len)!=3) 


0

In [85]:
data = data.assign(chaoxiang=data.cjlouceng.map(lambda x:x.split('/')[0])) 

In [86]:
data = data.assign(louceng=data.cjlouceng.map(lambda x:x.split('/')[1]))


In [87]:
del data['cjlouceng']

In [88]:
top15 = data.xiaoqu.value_counts()[:15].index

In [89]:
data = data[data.xiaoqu.isin(top15)]

In [90]:
data['mianji'] = data.mianji.str.replace('平', '').astype(np.float32)

In [91]:
data = data[data.chaoxiang!='暂无数据']

In [92]:
data.huxing.unique()

array(['1室1厅', '1室--厅', '2室1厅', '2室2厅', '3室2厅', '4室2厅', '1室2厅', '3室1厅',
       '5室1厅', '3室--厅', '5室--厅', '4室1厅', '5室2厅', '3室3厅', '2室--厅', '2室3厅',
       '4室--厅'], dtype=object)

In [93]:
data.huxing = data.huxing.str.replace('--','1')

In [94]:
# 独热编码 one-hot （有那个属性，就标记为1，否则为0，为了防止属性之间存在线性关系）
data = data.join(pd.get_dummies(data[['xiaoqu', 'huxing', 'louceng']]))

In [95]:
data.chaoxiang.unique()

array(['南', '东', '南 北', '东 西', '南 西', '南 北 西', '西南', '西 北', '东南', '东 北',
       '东 南', '西 南', '东北', '东 南 北', '西北', '西', '北', '西南 东北', '西 南 北',
       '东 南 西', '南 北 东', '南 西 北', '东 西 北', '东 西 南', '北 东南', '北 西南',
       '东 北 南'], dtype=object)

In [96]:
data['dong'] = (data.chaoxiang.map(lambda x:'东' in x.split())).astype(np.int32)
data['xi'] = (data.chaoxiang.map(lambda x:'西' in x.split())).astype(np.int32)
data['nan'] = (data.chaoxiang.map(lambda x:'南' in x.split())).astype(np.int32)
data['bei'] = (data.chaoxiang.map(lambda x:'北' in x.split())).astype(np.int32)
data['dongnan'] = (data.chaoxiang.map(lambda x:'东南' in x.split())).astype(np.int32)
data['dongbei'] = (data.chaoxiang.map(lambda x:'东北' in x.split())).astype(np.int32)
data['xinan'] = (data.chaoxiang.map(lambda x:'西南' in x.split())).astype(np.int32)
data['xibei'] = (data.chaoxiang.map(lambda x:'西北' in x.split())).astype(np.int32)


In [97]:
data.head()

,cjdanjia,xiaoqu,huxing,mianji,chaoxiang,louceng,xiaoqu_东亚上北中心,xiaoqu_北京像素北区,xiaoqu_北京像素南区,xiaoqu_北京新天地,...,louceng_地下室,louceng_高楼层,dong,xi,nan,bei,dongnan,dongbei,xinan,xibei
31,6.62,荣丰2008,1室1厅,32.0,南,低楼层,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
148,3.86,芍药居北里,1室1厅,43.0,南,地下室,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
260,3.81,沿海赛洛城,1室1厅,64.0,东,低楼层,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
261,3.32,沿海赛洛城,1室1厅,57.0,东,中楼层,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
285,3.96,沿海赛洛城,2室2厅,105.0,南 北,中楼层,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [98]:
data.drop(data.columns[[1, 2, 4, 5]], axis=1, inplace=True)

In [99]:
Y = data.cjdanjia
X = data[[x for x in data.columns if x != 'cjdanjia']]


In [100]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [101]:
np.shape(X_train.values)

(1389, 40)

In [102]:
model = LinearRegression()

In [103]:
model.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

模型评价

In [105]:
metrics.mean_squared_error(model.predict(X_test), Y_test)

0.29632991770529843

In [108]:
data_test = pd.Series(np.zeros(len([x for x in data.columns if x != 'cjdanjia'])), index=[x for x in data.columns if x != 'cjdanjia'])

In [117]:
# 构造待预测的数据
data_test.mianji = 80
data_test.xiaoqu_天通西苑三区 = 1
data_test.nan = 1
data_test.bei = 1
data_test.huxing_2室2厅 = 1
data_test.louceng_地下室 = 1

In [118]:
model.predict(data_test.values.reshape(1, -1))


array([1.28826657])